In [3]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy import spatial
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from hdx.data.resource import Resource
import numpy as np
import nltk
import re 
import gensim
import os;
import re 
import pandas as pd
import re;
import logging;
import sqlite3;
import time;
import sys;
import multiprocessing;
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt;
from itertools import cycle;
from io import StringIO

import pandas as pd
import requests

In [1]:
import warnings


warnings.filterwarnings('ignore')

In [4]:

import Utility

In [62]:
## Define important paramters:
Num_of_words = 20 ## 15, 20 , 30
max_DF =0.90 ## 0.85 , 0.95
k = 4 # the extra tags returned by the model.



In [6]:
# read the preprocessed content:
# a list of datasets and their metadata
df=pd.read_csv("after_clean_v3.csv")



In [11]:
df.columns

Index(['title', 'notes', 'tags', 'header', 'organization', 'dataset_source',
       'geodata', 'country', 'tag_list', 'doc'],
      dtype='object')

In [7]:
# do cleaning for the header if it was not done
df['header'] = Utility.data_clean(df,['header'])

In [13]:
# concatenate the content of all metadata fields into raw text
def get_all_text(df):
    #df.tags.astype(str)+ " " +
    textdata =  df.title.fillna('').astype(str)+" " + df.header.fillna('').astype(str) +" " + df.organization.fillna('').astype(str) +" " +  df.notes.fillna('').astype(str)+" "+ df.country.fillna('').astype(str) + " " + df.geodata.fillna('').astype(str) 
    return textdata

In [8]:
# define the stop words list in addition standard english stopwrds
stoplist= set(stopwords.words('english'))
stoplist.update(['type','etc','also','whole','this','can','be','unnamed','nan','file','xls','xlsx','etc','zip','link',
                 'description','https','of','name'])

In [14]:
# concatenate the text of all metadata fields
df['All_text'] =  get_all_text (df)

In [15]:
# remove the stopwords 
df['text']=  Utility.remove_stopwords(df['All_text'],stoplist)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
import re
  
#load a set of stop words
#get the text column 
docs=df['All_text'].tolist()
 
#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.90,stop_words=stoplist,binary=True)#, ngram_range =(1,2))
word_count_vector=cv.fit_transform(docs)

In [17]:
# term frequncy inverse document frequncy transformer  
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=False)
tfidf_transformer.fit(word_count_vector)
 

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False,
         use_idf=False)

In [18]:
TfidfTransformer.get_params

<function sklearn.base.BaseEstimator.get_params(self, deep=True)>

In [19]:
##quick look at top words
list(cv.vocabulary_.keys())[:3]

['somalia', 'demographic', 'health']

In [27]:
# a mapping of index to word name
feature_names=cv.get_feature_names()

In [23]:
## use this as a test to see the top words in the docuemnt
# get the document that we want to extract keywords from
doc=df['All_text'].ix[100]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) #tfidf_transformer
 
#sort the tf-idf vectors by descending order of scores
sorted_items=Utility.sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=Utility.extract_topn_from_vector(feature_names,sorted_items,30)
 
# now print the results
print("\n=====Doc=====")
#print(doc)
print(" current tags " , df['tags'].ix[100])

print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
 current tags  demographics education indicators socioeconomics sustainable development

===Keywords===
venezuela 0.258
various 0.258
unesco 0.258
sustainable 0.258
socioeconomic 0.258
republic 0.258
portal 0.258
indicators 0.258
education 0.258
development 0.258
demographic 0.258
data 0.258
covering 0.258
contains 0.258
bolivarian 0.258


/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [28]:
# get the document that we want to extract keywords from.
#generate tf-idf for the given document.
# sort the tf-idf vectors by descending order of scores.
#extract only the top n; n here is 10

tfidf_vecs = []
df['tfidf'] =" "

for i, row in df.iterrows():
    doc=df['All_text'].ix[i]
    candidates=""
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) #tfidf_transformer.transform(
    sorted_items=Utility.sort_coo(tf_idf_vector.tocoo()) 
    keywords=Utility.extract_topn_from_vector(feature_names,sorted_items,Num_of_words)
    vec =np.array(list(keywords.values()))
    for k in keywords: 
        candidates+=k +" "
    tfidf_vecs.append(vec)
    df.ix[i]['tfidf'] = candidates  
 

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [29]:
df['text'].ix[20]

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'somalia drought dashboard somalia drought dashboard  dataset contains data used humanitarian country team somalia monitor evolution drought situation somalia data covers following topics internal displacement cause drought related conflict insecurity cause river levels shabelle juba rivers water prices region cumulative annual rainfall disease burden acute watery diarrhea cholera bloody diarrhea measles monthly response monitoring region following indicators cccm number people benefiting site improvement projects education number children access safe drinking water food security number people reached activities geared towards improving access food safety nets health number medical consultations nutrition number acute malnutrition admissions protection number girls boys women participating community based psycho social activities shelter number people need emergency assistance receiving appropriate nfis kind distribution vouchers cash wash number people reached access sustainable safe 

In [30]:
df['tfidf'].ix[20]

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


'women watery water wash vouchers used unhcr towards topics team swalim sustainable sources somalia social situation site shelter shabelle services '

In [36]:
# Google pretrained word2vec model:
model, word = Utility.get_model()
index2word_set = set(model.wv.index2word)

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
## read the list of HDX valid tags
tag_list_clean = Utility.read_concept_tags()
len(tag_list_clean)

364

In [37]:
## do tag expansion using word2vec simialr words
new_tag_list_clean = []
for tag in tag_list_clean:
    expanded = Utility.expan_tags(tag, model , 5)
    if expanded =="":
        new_tag_list_clean.append(tag)
    else:
        new_tag_list_clean.append(expanded+" "+tag)
    

In [38]:
## example of cosine distance
Utility.cosine_distance(df['tfidf'].ix[20], tag_list_clean,model,index2word_set,10)

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/Users/dewet/Downloads/hdx-work/Utility.py:332: RuntimeWarning: invalid value encountered in float_scalars
  cos_sim = np.dot(doc, tag)/(np.linalg.norm(doc)*np.linalg.norm(tag))


[('water points - water sources', 0.5816562),
 ('water sanitation and hygiene - wash', 0.57284635),
 ('access to water', 0.5491847),
 ('people targeted for assistance', 0.54868484),
 ('water bodies - hydrography', 0.5279947),
 ('displaced persons locations - camps - shelters', 0.5189776),
 ('food security', 0.49767295),
 ('aid workers security', 0.49478364),
 ('access to sanitation', 0.49318522),
 ('social development centers', 0.48911437)]

In [39]:
# 20% of the datasets that was prepared 
#for the evaluation of keyword extraction

evaluation_set = pd.read_csv('good_test.csv')

In [40]:
'''
This code snippets- for each dataset:
 1) get the document that we want to extract keywords from
 2) generate tf-idf for the document
 3) sort the tf-idf vectors by descending order of scores
 4) extract only the top n; n here is 30
 5) clean the content (number, special character,links and/or non-english chars removal )
 6) concatenate the text content from all metdata fields.
'''
evaluation_set['tfidf'] =""
for i, row in evaluation_set.iterrows():
    doc=evaluation_set.ix[i]['All_text']
    candidates=""
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc])) #tfidf_transformer.transform(
    sorted_items=Utility.sort_coo(tf_idf_vector.tocoo()) 
    keywords=Utility.extract_topn_from_vector(feature_names,sorted_items,Num_of_words)
    for k in keywords:
        candidates+=k +" "
    evaluation_set.at[i,'tfidf'] = candidates      

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [41]:
## convert the string of list (tags) to list of (tags) 
#problem for reading list from CSV
from ast import literal_eval
evaluation_set.loc[:,'perfect_tags'] = evaluation_set.loc[:,'perfect_tags'].apply(lambda x: literal_eval(x))

In [42]:
# Actual list: is the list of tags currenly assigned to the dataset by the providers.
# pred_list: The list of tags suggested by the model.
#tfidf_list: list of keywords extracted by tfidf
# tf_listL list of keywords extracted by term frequncy
# perfect_list: list of valid tags.
actual_flist= evaluation_set['tag_list']
pred_list= evaluation_set['rec']
tfidf_list = evaluation_set['tfidf']
tf_list = evaluation_set['tf']
perfect_list = evaluation_set['perfect_tags']

In [43]:
# compare top words with the perfect list:
# an idea to overcome the exact matching.
result_tfidf=[]
for document_lst, pred_tags in zip(perfect_list,tfidf_list):
    matching= [s for s in document_lst if any(s in xs or xs in s for xs in [pred_tags])]
    result_tfidf.append((len(matching), len(document_lst)))
        

In [44]:
### compuate the recall for the keywords. 

tfidf_recall = []
for t in result_tfidf:
    tfidf_recall.append(t[0]/t[1])
np.average(tfidf_recall)

0.19292929292929295

In [45]:
## Optional - tag cleaning
P_tags =  Utility.clean_tag(new_tag_list_clean)
good_tags = Utility.remove_stopwords(pd.Series(P_tags), stoplist)

In [46]:
## example of cosine distance
Utility.cosine_distance("secondary data secondary nigeria data sdr review sdr review nigeria education education secondary data review sdr secondary sdr nigeria matrix nigeria matrix education nigeria education education data matrix", tag_list_clean,model,index2word_set,10)

[('nigeria - complex emergency - 2014-', 0.6186349),
 ('education', 0.5417682),
 ('education cluster', 0.5060455),
 ('central africa', 0.48183042),
 ('educational attainment', 0.47909716),
 ('access to education', 0.4781804),
 ('education facilities', 0.47800997),
 ('sex and age disaggregated data - sadd', 0.46941486),
 ('social media data', 0.452918),
 ('africa', 0.4403102)]

In [63]:
# run the model on the evaluation set 
# represent the tfidf of the documents into word2vec. 
# Find top tags from the clean_list to each dataset using the 
df['rec'] = ""
for i, row in df.iterrows():
    doc = df.ix[i]['tfidf']
    #if i ==12:
     #   print(doc)
    suggested_tags =Utility.cosine_distance_list(doc, tag_list_clean,model,index2word_set,k )#evaluation_set.ix[i]['k']+5) 
    #print(suggested_tags)
    df.at[i, 'rec'] = suggested_tags

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [55]:
##final list
## represent the tfidf of the documents into word2vec
##recommend tags from the hdx valid tag list to each dataset based on cosine distance 
for i, row in evaluation_set.iterrows():
    doc = evaluation_set.ix[i]['tfidf']
    suggested_tags =Utility.cosine_distance_list(doc, good_tags,model,index2word_set,evaluation_set.ix[i]['k']+5)
    #print(suggested_tags)
    evaluation_set.at[i, 'rec'] = suggested_tags

/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/Users/dewet/anaconda/envs/hdx/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [56]:
pred_list= evaluation_set['rec']

In [57]:
####compare word2vec of top words with the perfect list:
## an idea for the comparsion
res_word2vec=[]
total = 0
for document_lst, pred_tags in zip(perfect_list,pred_list):
    matching= [s for s in document_lst if any(s in xs or xs in s for xs in pred_tags)]
    print("document_lst", document_lst)
    print("predicted", pred_list)
    res_word2vec.append((len(matching), len(document_lst)))
        

document_lst ['education', 'health', 'baseline population population', 'transportation transport']
predicted 0     [population movement, alan broadbent expert ur...
1     [population movement, alan broadbent expert ur...
2     [population movement, alan broadbent expert ur...
3     [reproductive health family planning, health s...
4     [population movement, alan broadbent expert ur...
5     [socioeconomic status socioeconomic ethnicity ...
6     [population movement, education cluster, commo...
7     [socioeconomic status socioeconomic ethnicity ...
8     [population movement, alan broadbent expert ur...
9     [population movement, alan broadbent expert ur...
10    [socioeconomic status socioeconomic ethnicity ...
11    [sex age disaggregated data sadd, socioeconomi...
12    [population movement, alan broadbent expert ur...
13    [education cluster, health districts, educatio...
14    [sri lankans somali dafur ltte ethiopia darfur...
15    [gross national income gni, gross domestic pr

predicted 0     [population movement, alan broadbent expert ur...
1     [population movement, alan broadbent expert ur...
2     [population movement, alan broadbent expert ur...
3     [reproductive health family planning, health s...
4     [population movement, alan broadbent expert ur...
5     [socioeconomic status socioeconomic ethnicity ...
6     [population movement, education cluster, commo...
7     [socioeconomic status socioeconomic ethnicity ...
8     [population movement, alan broadbent expert ur...
9     [population movement, alan broadbent expert ur...
10    [socioeconomic status socioeconomic ethnicity ...
11    [sex age disaggregated data sadd, socioeconomi...
12    [population movement, alan broadbent expert ur...
13    [education cluster, health districts, educatio...
14    [sri lankans somali dafur ltte ethiopia darfur...
15    [gross national income gni, gross domestic pro...
16    [exchange rates, gross national income gni, gr...
17    [transporation transport joe mal

predicted 0     [population movement, alan broadbent expert ur...
1     [population movement, alan broadbent expert ur...
2     [population movement, alan broadbent expert ur...
3     [reproductive health family planning, health s...
4     [population movement, alan broadbent expert ur...
5     [socioeconomic status socioeconomic ethnicity ...
6     [population movement, education cluster, commo...
7     [socioeconomic status socioeconomic ethnicity ...
8     [population movement, alan broadbent expert ur...
9     [population movement, alan broadbent expert ur...
10    [socioeconomic status socioeconomic ethnicity ...
11    [sex age disaggregated data sadd, socioeconomi...
12    [population movement, alan broadbent expert ur...
13    [education cluster, health districts, educatio...
14    [sri lankans somali dafur ltte ethiopia darfur...
15    [gross national income gni, gross domestic pro...
16    [exchange rates, gross national income gni, gr...
17    [transporation transport joe mal

predicted 0     [population movement, alan broadbent expert ur...
1     [population movement, alan broadbent expert ur...
2     [population movement, alan broadbent expert ur...
3     [reproductive health family planning, health s...
4     [population movement, alan broadbent expert ur...
5     [socioeconomic status socioeconomic ethnicity ...
6     [population movement, education cluster, commo...
7     [socioeconomic status socioeconomic ethnicity ...
8     [population movement, alan broadbent expert ur...
9     [population movement, alan broadbent expert ur...
10    [socioeconomic status socioeconomic ethnicity ...
11    [sex age disaggregated data sadd, socioeconomi...
12    [population movement, alan broadbent expert ur...
13    [education cluster, health districts, educatio...
14    [sri lankans somali dafur ltte ethiopia darfur...
15    [gross national income gni, gross domestic pro...
16    [exchange rates, gross national income gni, gr...
17    [transporation transport joe mal

predicted 0     [population movement, alan broadbent expert ur...
1     [population movement, alan broadbent expert ur...
2     [population movement, alan broadbent expert ur...
3     [reproductive health family planning, health s...
4     [population movement, alan broadbent expert ur...
5     [socioeconomic status socioeconomic ethnicity ...
6     [population movement, education cluster, commo...
7     [socioeconomic status socioeconomic ethnicity ...
8     [population movement, alan broadbent expert ur...
9     [population movement, alan broadbent expert ur...
10    [socioeconomic status socioeconomic ethnicity ...
11    [sex age disaggregated data sadd, socioeconomi...
12    [population movement, alan broadbent expert ur...
13    [education cluster, health districts, educatio...
14    [sri lankans somali dafur ltte ethiopia darfur...
15    [gross national income gni, gross domestic pro...
16    [exchange rates, gross national income gni, gr...
17    [transporation transport joe mal

In [58]:
word2vec_recall = []
for t in res_word2vec:
    word2vec_recall.append(t[0]/t[1])
print(np.mean(word2vec_recall))

0.549074074074074
